In [1]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests
from scrapy.selector import Selector
from tqdm import tqdm
import re

In [9]:
def email_from_web(X):
    try:
        url=[]
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36"}
        web=str('http://www.'+str(X))
        url.append(web)
        #-------------fetch all the links from website and filtter only contect and about link----------------------
        links=[]
        req = Request(web,headers=headers)
        html_page = urlopen(req)
        soup = BeautifulSoup(html_page, "lxml")
        for link in soup.findAll('a'):
            links.append(link.get('href'))
        #-----------------------------------------------------------------------------------------------------------    
        url_list=[]
        for i in range(0,len(links),1):
            if "contact" in str(links[i]).lower():
                url_list.append(web+str(links[i]))
            if "about" in str(links[i]).lower():
                url_list.append(web+str(links[i]))
        #--------------------------------------------------- ---------------------------------------------------------         
        #print(url_list)
        for i in url_list:
            if str(web[-3:]+'http') in i:
                url.append('http'+i.split(web[-3:]+'http')[1])
            if i not in url: 
                url.append(i)   
        #print(url)        
        #--------------------------section 2 get response and fetch data-----------------------------------------------
        emails=[]
        for i in tqdm(url):
            try:
                response=requests.get(i,headers=headers)
            except:
                  pass
#                 print('response time error')
            if str(response)=="<Response [200]>":
                #print(i)
                sel=Selector(text=response.text)
                email_1=sel.xpath('//a[contains(@href,"mailto")]/@href').extract()
                email_2=sel.xpath('//*[contains(text(),"info@")]/text()').extract()
                email_3=sel.xpath('//*[contains(text(),"@gmail")]/text()').extract_first()
                email_4=sel.xpath('//*[contains(text(),"@aol")]/text()').extract_first()
                email_5=sel.xpath('//*[contains(text(),"@hotmail")]/text()').extract_first()
                email_6=sel.xpath('//*[contains(text(),"@yahoo")]/text()').extract_first()
                
                #---------------section 3 store into list-------------------------------
                emails.extend(email_1)
                emails.extend(email_2)
                emails.append(email_3)
                emails.append(email_4)
                emails.append(email_5)
                emails.append(email_6)    
        #----------------preprocess the data and return Email if its present------------------------------------------
        res = []    
        for val in emails: 
            if val != None : 
                res.append(val) 
        if res:        
            for i in res:        
                match = re.findall(r'([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)',i)
                for i in match:
                    return i
        else:
            return "None"
        #----------------------------------END-----------------------------------------------    
    except Exception as e:
        pass

In [10]:
email_from_web('healthcaresuccess.com')

100%|██████████████████████████████████████████| 12/12 [00:19<00:00,  1.60s/it]


'Kyle@HealthcareSuccess.com'